In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from IPython.display import clear_output
from astromy_ds9 import ds9_norm
import openpyxl

plt.ioff()

In [ ]:
# Opens FITS files and extracts their information.
def open_file(path):
    hdu_rgb = fits.open(path + "/" + path + "_ds9.fits")
    hdu_i3 = fits.open(path + "/" + path + "_i3.fits")
    
    # Extract header information.
    wcs = WCS(hdu_rgb[0].header)
    # Extract and return RGB data.
    return hdu_rgb[1].data, hdu_rgb[2].data, hdu_rgb[3].data, hdu_i3[0].data, wcs

In [ ]:
# Read scale and colormap parameters of the image from "param.txt".
# The parameter file has the following format:
#   path
#   red_low_limit
#   red_high_limit
#   green_low_limit
#   green_high_limit
#   blue_low_limit
#   blue_high_limit
#   i3_low_limit
#   i3_high_limit
#   red_contrast
#   red_bias
#   green_contrast
#   green_bias
#   blue_contrast
#   blue_bias
#   i3_contrast
#   i3_bias

# Function returns a list of the read parameters.
def read_params(path):
    param_file = open("param.txt")
    lst = param_file.read().split()
    # Find index of the line which contains the name of the region we're analyzing.
    i = lst.index(path)
    # Create a sub-list with the image parameters and convert all the strings to floats.
    new_lst = (float(j) for j in lst[i+1:i+17])
    return new_lst

In [ ]:
# Applies the scale and colormap parameters to the images.
def update_image(RedData, GreenData, BlueData, I3Data, params):
    # Set given parameters.
    red_low_limit, red_high_limit, green_low_limit, green_high_limit, blue_low_limit, blue_high_limit, i3_low_limit, i3_high_limit, red_contrast, red_bias, green_contrast, green_bias, blue_contrast, blue_bias, i3_contrast, i3_bias = params

    # Apply scaling to each channel to emphasize peaks and reduce noise.
    Rednorm = ds9_norm(vmin=red_low_limit, vmax=red_high_limit, bias=red_bias, contrast=red_contrast, stretch='sqrt')(RedData)
    Greennorm = ds9_norm(vmin=green_low_limit, vmax=green_high_limit, bias=green_bias, contrast=green_contrast, stretch='sqrt')(GreenData)
    Bluenorm = ds9_norm(vmin=blue_low_limit, vmax=blue_high_limit, bias=blue_bias, contrast=blue_contrast, stretch='sqrt')(BlueData)
    I3norm = ds9_norm(vmin=i3_low_limit, vmax=i3_high_limit, bias=i3_bias, contrast=i3_contrast, stretch='sqrt')(I3Data)

    # Stack the normalized channels.
    rgb_image = np.dstack((Rednorm, Greennorm, Bluenorm))

    # Create the RGB image
    plt.figure(figsize=(10, 10))
    # Option to display the image.
    # plt.imshow(rgb_image, origin='lower')
    # plt.show()
    
    # Return the updated (scaled) RGB data.
    return Rednorm, Greennorm, Bluenorm, I3norm

In [ ]:
# Creates a figure of a given (single) filter covering a region in the image.
# The regions are determined by the .reg files.
def create_region(x_coord, y_coord, image, wcs, filtro, half_size=100):    
    # Ensure the provided coordinates are in pixel units.
    int_x_coord = int(x_coord)
    int_y_coord = int(y_coord)

    # Define the coordinates of the center of the region.
    center_x = half_size
    center_y = half_size

    # Create a figure and axis.
    px = 1 / plt.rcParams["figure.dpi"] # Pixel in inches.
    fig, ax = plt.subplots(figsize=(256 * px, 256 * px), subplot_kw={'projection': wcs})
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0, hspace=0)
    plt.margins(0, 0)
    ax.set_axis_off()

    # Extract the region using WCS information from the red channel.
    cutout = Cutout2D(image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)

    # Display the region extracted with WCS.
    ax.imshow(cutout.data, origin='lower')

    # Disable axis labels and ticks.
    ax.coords.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Option to display the plot.
    print(filtro)
    # plt.show()

    return fig

In [ ]:
# Creates an overlay of the 3 filters to produce an RGB image.
def overlayFilters(x_coord, y_coord, red_image, green_image, blue_image, wcs, filtro, half_size=100):
    # Ensure the provided coordinates are in pixel units.
    int_x_coord = int(x_coord)
    int_y_coord = int(y_coord)

    # Define the coordinates of the center of the region.
    center_x = half_size
    center_y = half_size

    # Create a figure and axis.
    px = 1 / plt.rcParams["figure.dpi"] # Pixel in inches.
    fig, ax = plt.subplots(figsize=(256 * px, 256 * px), subplot_kw={'projection': wcs})
    fig.subplots_adjust(top=1, bottom=0, left=0, right=1, wspace=0, hspace=0)
    plt.margins(0, 0)
    ax.set_axis_off()

    # Extract the regions using WCS information from the different filters.
    red_cutout = Cutout2D(red_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)
    green_cutout = Cutout2D(green_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)
    blue_cutout = Cutout2D(blue_image, (int_x_coord, int_y_coord), (2 * half_size, 2 * half_size), wcs=wcs)

    # Combine the filters into a composite RGB image.
    composite_image = np.dstack((red_cutout.data, green_cutout.data, blue_cutout.data))

    # Display the composite image with WCS.
    ax.imshow(composite_image, origin='lower')

    # Disable axis labels and ticks.
    ax.coords.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Option to display the plot.
    print(filtro)
    # plt.show()
    
    return fig

In [ ]:
def read_region_files(path):
    # Reads region file in physical coordinates.
    phys_file = path + '/fisicas_' + path + '.reg'
    # Reads region file in galactic coordinates.
    gal_file = path + '/galacticas_' + path + '.reg'
    # Return filenames. Files will be opened in "output_region".
    return phys_file, gal_file

In [ ]:
# Write images into spreadsheet. This function will only be used for testing purposes.
def put_images(picsheet, rgb_name, r_name, g_name, b_name, i3_name, src_id):
    rgb_img = openpyxl.drawing.image.Image(rgb_name)
    r_img = openpyxl.drawing.image.Image(r_name)
    g_img = openpyxl.drawing.image.Image(g_name)
    b_img = openpyxl.drawing.image.Image(b_name)
    i3_img = openpyxl.drawing.image.Image(i3_name)

    picsheet["A" + str(tile)] = src_id
    picsheet.add_image(rgb_img, "B" + str(tile))
    picsheet.add_image(r_img, "F" + str(tile))
    picsheet.add_image(i3_img, "J" + str(tile))
    picsheet.add_image(g_img, "N" + str(tile))
    picsheet.add_image(b_img, "R" + str(tile))

In [ ]:
# Final function that appends the data to an Excel spreadsheet.
tile = 2
def output_region(path, r_scale, g_scale, b_scale, i3_scale, phys_reg, gal_reg, worksheet, picsheet, workbook, wcs, count):
    global tile
    with open(phys_reg, 'r') as f_physical:
        lines_physical = f_physical.readlines()

    with open(gal_reg, 'r') as f_galactic:
        lines_galactic = f_galactic.readlines()

    # Iterate over each line in the physical coordinate file.
    for line_physical, line_galactic in zip(lines_physical, lines_galactic):
        # Remove leading/trailing whitespace and skip empty lines.
        line_physical = line_physical.strip()
        line_galactic = line_galactic.strip()
        
        # Extract the physical and Galactic coordinates.
        physical_parts = line_physical.split()
        galactic_parts = line_galactic.split()
        
        physical_x_str, physical_y_str = physical_parts[0], physical_parts[1]
        x = float(physical_x_str)
        y = float(physical_y_str)

        galactic_x_str, galactic_y_str = galactic_parts[0], galactic_parts[1]
        galactic_x = float(galactic_x_str)
        galactic_y = float(galactic_y_str)
                
        # Print the current field and the Galactic coordinates.
        print("Field:")
        print(path)
        print()
        print("Galactic Coordinates:")
        print(galactic_x_str, galactic_y_str)
        print()

        # Functions to display images.
        figure = overlayFilters(x, y, r_scale, g_scale, b_scale, wcs, "RGB", half_size=50)

        # Save figure into image.
        rgb_name = "cuts/" + galactic_x_str + "," + galactic_y_str + "_" + path + "_RGB-composite.jpeg"
        r_name = "cuts/" + galactic_x_str + "," + galactic_y_str + "_" + path + "_Rojo-8micras.jpeg"
        g_name = "cuts/" + galactic_x_str + "," + galactic_y_str + "_" + path + "_Verde-4.6micras.jpeg"
        b_name = "cuts/" + galactic_x_str + "," + galactic_y_str + "_" + path + "_Azul-3.5micras.jpeg"
        i3_name = "cuts/" + galactic_x_str + "," + galactic_y_str + "_" + path + "_I3-5.8micras.jpeg"

        plt.savefig(rgb_name)
        fig1 = create_region(x, y, r_scale, wcs, "Rojo-8 micras", half_size=50)
        plt.savefig(r_name)
        fig2 = create_region(x, y, g_scale, wcs, "Verde-4.6 micras", half_size=50)
        plt.savefig(g_name)
        fig3 = create_region(x, y, b_scale, wcs, "Azul-3.5 micras", half_size=50)
        plt.savefig(b_name)
        fig4 = create_region(x, y, i3_scale, wcs, "5.8 micras", half_size=50)
        plt.savefig(i3_name)

        # Write data into Excel spreadsheet.
        src_id = path + "_" + str(count)
        worksheet.append([src_id, galactic_x, galactic_y, "Yes", rgb_name, r_name, i3_name, g_name, b_name])

        # Option to write images into spreadsheet.
        # put_images(picsheet, rgb_name, r_name, g_name, b_name, i3_name, src_id)

        workbook.save("output.xlsx")

        # Close the image.
        plt.close("all");
        count += 1
        tile += 13

        # Clear the output cell in Jupyter Notebook
        clear_output()

In [ ]:
# Set identifiers for all SMOG regions to be analyzed.
filenames = ["10050_0270",
             "10150_0270",
             "10250_0145",
             "10250_0270",
             "10350_0020",
             "10350_0145",
             "10350_0270",
             "10550_0020",
             "10550_0270"]

# Temporary list of one region for testing purposes.
# filenames = ["10050_0270"]

# Initialize Excel file.
workbook = openpyxl.Workbook()
worksheet = workbook.worksheets[0]
count = 1
# Temporary sheet for images to aid visualizations.
workbook.create_sheet()
picsheet = workbook.worksheets[1]

worksheet.append(["ID",
        "Galactic X",
        "Galactic Y",
        "Galaxy?",
        "RGB Image",
        "8-micron Filter",
        "5.8-micron Filter",
        "4.6-micron Filter",
        "3.5-micron Filter"])

picsheet.append(["ID", "RGB Image", "", "", "", "8-micron Filter", "", "", "", "5.8-micron Filter", "", "", "", "4.6-micron Filter", "", "", "", "3.5-micron Filter"])

# Iterate through and produce data for all regions.
for path in filenames:
    RedData, GreenData, BlueData, I3Data, wcs = open_file(path)
    params = read_params(path)
    new_RedScaled, new_GreenScaled, new_BlueScaled, new_I3Scaled = update_image(RedData, GreenData, BlueData, I3Data, params)
    phys_reg, gal_reg = read_region_files(path)
    output_region(path, new_RedScaled, new_GreenScaled, new_BlueScaled, new_I3Scaled, phys_reg, gal_reg, worksheet, picsheet, workbook, wcs, count)

clear_output()